In [1]:
!git clone https://github.com/RulinShao/rag-evaluation-harness.git
%cd rag-evaluation-harness
!conda create -n lm-eval python=3.11
!pip install -e .
!mkdir -p /content/rag-evaluation-harness/lm_eval/tasks/sysengbench
!touch /content/rag-evaluation-harness/lm_eval/tasks/sysengbench/demo_sysengbench.yaml

Cloning into 'rag-evaluation-harness'...
remote: Enumerating objects: 3581, done.
remote: Counting objects: 100% (3581/3581), done.
remote: Compressing objects: 100% (2511/2511), done.
remote: Total 3581 (delta 1062), reused 3563 (delta 1046), pack-reused 0 (from 0)
Receiving objects: 100% (3581/3581), 1.74 MiB | 10.81 MiB/s, done.
Resolving deltas: 100% (1062/1062), done.
/content/rag-evaluation-harness
/bin/bash: line 1: conda: command not found
Obtaining file:///content/rag-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━

HuggingFace login

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Mounting Drive:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Custom lm_harness Task

In [4]:
YAML_SysEngBench_string = '''
task: demo_sysengbench
dataset_path: rabell/SysEngBench
dataset_name: null
dataset_kwargs: null

description: "The following are multiple-choice questions (with answers) related to System Engineering."

training_split: null
validation_split: null
test_split: test
fewshot_split: null

output_type: multiple_choice
doc_to_text: |
  Read the question below and select the best answer from the options provided. Respond with only the letter corresponding to the correct answer (A, B, C, or D).

  Question: {{question.strip()}}
  A. {{choiceA}}
  B. {{choiceB}}
  C. {{choiceC}}
  D. {{choiceD}}
  Answer:
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: label
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
'''

# Define the path where the YAML file will be written
rag_yaml_file_path = "/content/rag-evaluation-harness/lm_eval/tasks/sysengbench/demo_sysengbench.yaml"

# Write the YAML content to the file
with open(rag_yaml_file_path, "w") as f:
    f.write(YAML_SysEngBench_string)

print(f"YAML file written to {rag_yaml_file_path}")


YAML file written to /content/rag-evaluation-harness/lm_eval/tasks/sysengbench/demo_sysengbench.yaml


Bench mark subset consisting of 10% of the original questions (160)

In [5]:
# Define the path where the YAML file will be written
rag_yaml_file_path = "/content/rag-evaluation-harness/lm_eval/tasks/sysengbench/partial_sysengbench.yaml"
YAML_SysEngBench_string = '''
task: partial_sysengbench
dataset_path: arrow
dataset_kwargs:
  data_files:
    test: /content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/Datasets/partial_benchmark_start_at_5th_with_test_split/test/data-00000-of-00001.arrow

description: "The following are multiple-choice questions (with answers) related to System Engineering."

training_split: null
validation_split: null
test_split: test
fewshot_split: null

output_type: multiple_choice
doc_to_text: |
  Read the question below and select the best answer from the options provided. Respond with only the letter corresponding to the correct answer (A, B, C, or D).

  Question: {{question.strip()}}
  A. {{choiceA}}
  B. {{choiceB}}
  C. {{choiceC}}
  D. {{choiceD}}
  Answer:
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: label
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
'''

with open(rag_yaml_file_path, "w") as f:
    f.write(YAML_SysEngBench_string)


USED -YAML:

In [6]:
# Define the path where the YAML file will be written
# prev used - You are given a multiple-choice question. Your task is to read the provided context, think about whether it helps answer the question, and then select the best answer. Respond with only the letter corresponding to the correct answer (A, B, C, or D).
#  Read the question below and select the best answer from the options provided. Respond with only the letter corresponding to the correct answer (A, B, C, or D).
rag_yaml_file_path = "/content/rag-evaluation-harness/lm_eval/tasks/sysengbench/partial_sysengbench_prompt_eng.yaml"
YAML_SysEngBench_string = '''
task: partial_sysengbench_prompt_eng
dataset_path: arrow
dataset_kwargs:
  data_files:
    test: /content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/Datasets/partial_benchmark_start_at_1st_with_test_split/test/data-00000-of-00001.arrow

description: ""

training_split: null
validation_split: null
test_split: test
fewshot_split: null

output_type: multiple_choice
doc_to_text: |

  Answer the following Question.

  Question: {{question.strip()}}
  A. {{choiceA}}
  B. {{choiceB}}
  C. {{choiceC}}
  D. {{choiceD}}
  Answer:
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: label
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
'''

with open(rag_yaml_file_path, "w") as f:
    f.write(YAML_SysEngBench_string)

# importent!
there was an issue with the code that Chat gpt solved, needed to change line 520 in /content/rag-evaluation-harness/lm_eval/__main__.py to the following code block

In [ ]:
        #this!
        '''
        print(make_table(results, log_path=args.results_only_save_path))
        if "groups" in results:
            print(make_table(results, "groups", log_path=args.results_only_save_path))

        '''

        # change to this!
        '''
        if results is not None and "results" in results:
            print(make_table(results, log_path=args.results_only_save_path))

            if "groups" in results:
                print(make_table(results, "groups", log_path=args.results_only_save_path))
        else:
              print("No evaluation results generated because save_inputs_only is set to True.")

        '''




---



---



# **No RAG Evaluation Run:**

Models and Docs Constants:

In [16]:
# models
Mistral_7B="mistralai/Mistral-7B-v0.1"
Ministral_8B = "mistralai/Ministral-8B-Instruct-2410"
Ministral_3B = "ministral/Ministral-3b-instruct"
Orca_2_7B="microsoft/Orca-2-7b"
Llama_3_3B = "meta-llama/Llama-3.2-3B"
Llama_2_7B = "meta-llama/Llama-2-7b-chat-hf"
Llama_2_13B = "meta-llama/Llama-2-13b-chat-hf"
Gemma_2_2B = "google/gemma-2-2b-it"
Phi_3_4B = "microsoft/Phi-3-mini-4k-instruct"

# tasks
TASK="demo_sysengbench"
PARTIAL_TASK = "partial_sysengbench"
PARTIAL_TASK_PROMPT_ENG = "partial_sysengbench_prompt_eng"
INCLUDE_TASK_PATH = "/content/rag-evaluation-harness/lm_eval/tasks/"


# the retrived docs here use the most simple data split
# RETRIEVED_DOCS="/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/recursive_character_split_retrieved_documents_small_chunks.jsonl"

# retrived per task
# RETRIEVED_DOCS="/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/recursive_character_split_start_1_gpt_summerized_retrieved_documents.jsonl"
# RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/clues_framing_COT_retrieved_documents.jsonl"
# RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/recursive_paragraph_split_full_corpus_start_at_5_retrieved_documents.jsonl"
# RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/recursive_paragraph_split_full_corpus_retrieved_documents.jsonl"
# RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/recursive_paragraph_split_full_corpus_start_at_1_framing_retrieved_documents.jsonl"
# RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/recursive_paragraph_split_full_corpus_start_at_1_framing_COT_retrieved_documents.jsonl"
# RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/start_at_1st_new_rev_retrieved_documents.jsonl"
RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/start_at_1st_full_corpus_gpt_judge_no_empty_new_retrieved_documents.jsonl"
# RETRIEVED_DOCS = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/retrived_documents/partial_benchmark/start_at_5th_full_corpus_gpt_judge_no_empty_retrieved_documents.jsonl"


Llama_3_3B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_3_3B"
MISTRAL_7B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Mistral_7B"
ORCA_2_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Orca_2_7B"
Llama_2_7B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_2_7B"
Ministral_8B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Ministral_8B"
Llama_2_13B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_2_13B"
Ministral_3B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Ministral_3B"
Gemma_2_2B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Gemma_2_2B"
Phi_3_4B_OUTPUT_PATH = "/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Phi_3_4B"
################ NO RAG ################

llama_3_3b_no_RAG_results_path = Llama_3_3B_OUTPUT_PATH+"/no_RAG_results"
mistral_7b_no_RAG_results_path = MISTRAL_7B_OUTPUT_PATH+"/no_RAG_results"
orca_2_7b_no_RAG_results_path = ORCA_2_OUTPUT_PATH+"/no_RAG_results"
llama_2_7b_no_RAG_results_path = Llama_2_7B_OUTPUT_PATH+"/no_RAG_results"
ministral_8B_no_RAG_results_path = Ministral_8B_OUTPUT_PATH+"/no_RAG_results"
llama_2_13b_no_RAG_results_path = Llama_2_13B_OUTPUT_PATH+"/no_RAG_results"
ministral_3B_no_RAG_results_path = Ministral_3B_OUTPUT_PATH+"/no_RAG_results"
Gemma_2_2B_no_RAG_results_path = Gemma_2_2B_OUTPUT_PATH+"/no_RAG_results"
phi_3_4B_no_RAG_results_path = Phi_3_4B_OUTPUT_PATH+"/no_RAG_results"

# start at 1st
llama_3_3b_no_RAG_results_partial_1st_path = llama_3_3b_no_RAG_results_path + "/partial_1st_path"
mistral_7b_no_RAG_results_partial_1st_path = mistral_7b_no_RAG_results_path + "/partial_1st_path"
orca_2_7b_no_RAG_results_partial_1st_path = orca_2_7b_no_RAG_results_path + "/partial_1st_path"
llama_2_7b_no_RAG_results_partial_1st_path = llama_2_7b_no_RAG_results_path + "/partial_1st_path"
ministral_8B_no_RAG_results_partial_1st_path = ministral_8B_no_RAG_results_path + "/partial_1st_path"
llama_2_13b_no_RAG_results_partial_1st_path = llama_2_13b_no_RAG_results_path + "/partial_1st_path"
ministral_3B_no_RAG_results_partial_1st_path = ministral_3B_no_RAG_results_path + "/partial_1st_path"
gemma_2_2B_no_RAG_results_partial_1st_path = Gemma_2_2B_no_RAG_results_path + "/partial_1st_path"
phi_3_4B_no_RAG_results_partial_1st_path = phi_3_4B_no_RAG_results_path + "/partial_1st_path"

# start at 5th
mistral_7b_no_RAG_results_partial_5th_path = mistral_7b_no_RAG_results_path + "/partial_5th_path"
orca_2_7b_no_RAG_results_partial_5th_path = orca_2_7b_no_RAG_results_path + "/partial_5th_path"
llama_3_3b_no_RAG_results_partial_5th_path = llama_3_3b_no_RAG_results_path + "/partial_5th_path"
llama_2_7b_no_RAG_results_partial_5th_path = llama_2_7b_no_RAG_results_path + "/partial_5th_path"
ministral_8B_no_RAG_results_partial_5th_path = ministral_8B_no_RAG_results_path + "/partial_5th_path"
llama_2_13b_no_RAG_results_partial_5th_path = llama_2_13b_no_RAG_results_path + "/partial_5th_path"
ministral_3B_no_RAG_results_partial_5th_path = ministral_3B_no_RAG_results_path + "/partial_5th_path"
Gemma_2_2B_no_RAG_results_partial_5th_path = Gemma_2_2B_no_RAG_results_path + "/partial_5th_path"
phi_3_4B_no_RAG_results_partial_5th_path = phi_3_4B_no_RAG_results_path + "/partial_5th_path"


################ RAG ################
llama_2_7b_RAG_results_path = Llama_2_7B_OUTPUT_PATH+"/RAG_results"
mistral_7b_RAG_results_path = MISTRAL_7B_OUTPUT_PATH+"/RAG_results"
orca_2_7b_RAG_results_path = ORCA_2_OUTPUT_PATH+"/RAG_results"
llama_3_3b_RAG_results_path = Llama_3_3B_OUTPUT_PATH+"/RAG_results"
ministral_8B_RAG_results_path = Ministral_8B_OUTPUT_PATH+"/RAG_results"
llama_2_13b_RAG_results_path = Llama_2_13B_OUTPUT_PATH+"/RAG_results"
ministral_3B_RAG_results_path = Ministral_3B_OUTPUT_PATH+"/RAG_results"
Gemma_2_2B_RAG_results_path = Gemma_2_2B_OUTPUT_PATH+"/RAG_results"
phi_3_4B_RAG_results_path = Phi_3_4B_OUTPUT_PATH+"/RAG_results"

# start at 1st
llama_3_3b_RAG_results_partial_1st_path = llama_3_3b_RAG_results_path + "/partial_1st_path"
mistral_7b_RAG_results_partial_1st_path = mistral_7b_RAG_results_path + "/partial_1st_path"
orca_2_7b_RAG_results_partial_1st_path = orca_2_7b_RAG_results_path + "/partial_1st_path"
llama_2_7b_RAG_results_partial_1st_path = llama_2_7b_RAG_results_path + "/partial_1st_path"
ministral_8B_RAG_results_partial_1st_path = ministral_8B_RAG_results_path + "/partial_1st_path"
llama_2_13b_RAG_results_partial_1st_path = llama_2_13b_RAG_results_path + "/partial_1st_path"
ministral_3B_RAG_results_partial_1st_path = ministral_3B_RAG_results_path + "/partial_1st_path"
Gemma_2_2B_RAG_results_partial_1st_path = Gemma_2_2B_RAG_results_path + "/partial_1st_path"
phi_3_4B_RAG_results_partial_1st_path = phi_3_4B_RAG_results_path + "/partial_1st_path"

# start at 5th
orca_2_7b_RAG_results_partial_5th_path = orca_2_7b_RAG_results_path + "/partial_5th_path"
mistral_7b_RAG_results_partial_5th_path = mistral_7b_RAG_results_path + "/partial_5th_path"
llama_3_3b_RAG_results_partial_5th_path = llama_3_3b_RAG_results_path + "/partial_5th_path"
ministral_8B_RAG_results_partial_5th_path = ministral_8B_RAG_results_path + "/partial_5th_path"
llama_2_7b_RAG_results_partial_5th_path = llama_2_7b_RAG_results_path + "/partial_5th_path"
llama_2_13b_RAG_results_partial_5th_path = llama_2_13b_RAG_results_path + "/partial_5th_path"
gemma_2_2B_RAG_results_partial_5th_path = Gemma_2_2B_RAG_results_path + "/partial_5th_path"
phi_3__4B_RAG_results_partial_5th_path = phi_3_4B_RAG_results_path + "/partial_5th_path"


Query input extraction

In [ ]:
# Saves inputs based on given task to contect so we can download and move to drive - should be automatied in the feature.

!lm_eval --model hf \
    --model_args pretrained=$Llama_3_3B \
    --include_path /content/rag-evaluation-harness/lm_eval/tasks/ \
    --tasks $PARTIAL_TASK \
    --save_inputs_only \
    --inputs_save_dir /content


Traceback (most recent call last):
  File "/usr/local/bin/lm_eval", line 5, in <module>
    from lm_eval.__main__ import cli_evaluate
  File "/content/rag-evaluation-harness/lm_eval/__init__.py", line 1, in <module>
    from .evaluator import evaluate, simple_evaluate
  File "/content/rag-evaluation-harness/lm_eval/evaluator.py", line 13, in <module>
    import torch
  File "/usr/local/lib/python3.10/dist-packages/torch/__init__.py", line 368, in <module>
    from torch._C import *  # noqa: F403
  File "<frozen importlib._bootstrap>", line 216, in _lock_unlock_module
KeyboardInterrupt
^C


No RAG controll lm_hrness run Mistral_7B:

In [18]:
!lm_eval --model hf \
    --model_args pretrained=$Mistral_7B \
    --include_path $PARTIAL_TASK \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --output "{mistral_7b_no_RAG_results_partial_1st_path}"\
    --log_samples

2024-11-27 10:31:22.755441: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 10:31:22.772494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 10:31:22.793601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 10:31:22.799942: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 10:31:22.814925: I tensorflow/core/platform/cpu_feature_guar

No RAG controll lm_hrness run Orca_2_7B:

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=$Orca_2_7B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --output "{orca_2_7b_no_RAG_results_partial_5th_path}"\
    --log_samples

2024-11-15 15:22:22.782233: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 15:22:22.799438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 15:22:22.820403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 15:22:22.826787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 15:22:22.841851: I tensorflow/core/platform/cpu_feature_guar

No RAG controll lm_hrness run Llama_2_13B:

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=$Llama_2_13B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --output "{llama_2_13b_no_RAG_results_partial_5th_path}" \
    --log_samples


2024-11-15 15:24:37.601740: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 15:24:37.620295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 15:24:37.642576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 15:24:37.649145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 15:24:37.665078: I tensorflow/core/platform/cpu_feature_guar

No RAG controll lm_harness rus Gemma 2 2B:

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=$Gemma_2_2B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --output "{Gemma_2_2B_no_RAG_results_partial_5th_path}" \
    --log_samples

2024-11-21 10:21:08.038086: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 10:21:08.055131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 10:21:08.076441: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 10:21:08.082852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 10:21:08.098540: I tensorflow/core/platform/cpu_feature_guar

No RAG controll lm_harness rus Phi 3 3.8B:

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=$Phi_3_4B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --output "{phi_3_4B_no_RAG_results_partial_1st_path}" \
    --log_samples

2024-11-21 08:52:31.998769: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 08:52:32.015460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 08:52:32.036397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 08:52:32.042803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 08:52:32.058210: I tensorflow/core/platform/cpu_feature_guar

No RAG controll lm_harness rus Ministral 3B:

In [ ]:
# !lm_eval --model hf \
#     --model_args pretrained=$Ministral_3B \
#     --include_path $PARTIAL_TASK \
#     --tasks $PARTIAL_TASK_PROMPT_ENG \
#     --output "{ministral_3B_no_RAG_results_partial_1st_path}"\
#     --log_samples

2024-11-15 14:53:53.126936: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 14:53:53.143625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 14:53:53.164619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 14:53:53.170988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 14:53:53.185953: I tensorflow/core/platform/cpu_feature_guar

No RAG controll lm_hrness run Llama_3B:
- change model if needed

In [ ]:
# !lm_eval --model hf \
#     --model_args pretrained=$Llama_3_3B \
#     --include_path $INCLUDE_TASK_PATH \
#     --tasks $PARTIAL_TASK_PROMPT_ENG \
#     --output  "{llama_3_3b_no_RAG_results_partial_1st_path}"\
#     --log_samples

2024-11-15 14:06:24.709986: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 14:06:24.728015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 14:06:24.749562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 14:06:24.756060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 14:06:24.771914: I tensorflow/core/platform/cpu_feature_guar

No RAG controll lm_hrness run Llama_2_7B:

In [ ]:
# !lm_eval --model hf \
#     --model_args pretrained=$Llama_2_7B \
#     --include_path $INCLUDE_TASK_PATH \
#     --tasks $PARTIAL_TASK_PROMPT_ENG \
#     --output "{llama_2_7b_no_RAG_results_partial_1st_path}"\
#     --log_samples



2024-11-15 14:14:43.825486: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 14:14:43.843230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 14:14:43.864691: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 14:14:43.871222: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 14:14:43.886915: I tensorflow/core/platform/cpu_feature_guar



---



---



# **RAG Evaluating:**



---



---



RAG lm_harness run Mistral_7B::

In [17]:


# # Mistral as example:
# !lm_eval --model hf \
#     --model_args pretrained=$Mistral_7B \
#     --include_path $INCLUDE_TASK_PATH \
#     --tasks $OUR_TASK \
#     --retrieval_file "{RETRIEVED_DOCS}" \
#     --concat_k 2 \ # decides how many retrieved docs are added to system prompt.
#                    # this case it added the top 2 retrieved docs for this query
#     --output "{***********}" \
#     --log_samples

!lm_eval --model hf \
    --model_args pretrained=$Mistral_7B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --retrieval_file "{RETRIEVED_DOCS}" \
    --concat_k 2 \
    --output "{mistral_7b_RAG_results_partial_1st_path}" \
    --log_samples

# !lm_eval --model hf \
#     --model_args pretrained=$Gemma_2_2B \
#     --include_path $INCLUDE_TASK_PATH \
#     --tasks $PARTIAL_TASK_PROMPT_ENG \
#     --retrieval_file "{RETRIEVED_DOCS}" \
#     --concat_k 1 \
#     --output "{Gemma_2_2B_RAG_results_path}" \
#     --log_samples




2024-11-27 10:29:57.678852: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 10:29:57.695956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 10:29:57.716876: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 10:29:57.723155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 10:29:57.738334: I tensorflow/core/platform/cpu_feature_guar

RAG lm_harness run Llama_2_13B:

In [ ]:
#84.3 -> 81.2 -> 78.7
!lm_eval --model hf \
    --model_args pretrained=$Llama_2_13B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --retrieval_file "{RETRIEVED_DOCS}" \
    --concat_k 1 \
    --output "{llama_2_13b_RAG_results_partial_5th_path}" \
    --log_samples

2024-11-15 15:36:42.642941: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 15:36:42.660515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 15:36:42.681822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 15:36:42.688336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 15:36:42.703861: I tensorflow/core/platform/cpu_feature_guar

RAG lm_harness run Orca_2:

In [ ]:
#78.7 -> 81.2 ->86.8 -> 78 ->80 ->81.25
!lm_eval --model hf \
    --model_args pretrained=$Orca_2_7B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --retrieval_file "{RETRIEVED_DOCS}" \
    --concat_k 1 \
    --output "{orca_2_7b_RAG_results_partial_5th_path}" \
    --log_samples

2024-11-15 15:35:03.324240: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 15:35:03.342293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 15:35:03.364202: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 15:35:03.370735: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 15:35:03.386695: I tensorflow/core/platform/cpu_feature_guar

RAG lm_harness run Gemma 2 2B:

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=$Gemma_2_2B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --retrieval_file "{RETRIEVED_DOCS}" \
    --concat_k 1 \
    --output "{Gemma_2_2B_RAG_results_path}" \
    --log_samples

2024-11-21 10:24:35.644703: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 10:24:35.661672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 10:24:35.683064: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 10:24:35.689493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 10:24:35.705000: I tensorflow/core/platform/cpu_feature_guar

RAG lm_harness run Phi 3 3.8B::

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=$Phi_3_4B \
    --include_path $INCLUDE_TASK_PATH \
    --tasks $PARTIAL_TASK_PROMPT_ENG \
    --retrieval_file "{RETRIEVED_DOCS}" \
    --concat_k 1 \
    --output "{phi_3_4B_RAG_results_partial_1st_path}" \
    --log_samples

2024-11-21 10:13:44.670080: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 10:13:44.687242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 10:13:44.708517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 10:13:44.714929: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 10:13:44.730577: I tensorflow/core/platform/cpu_feature_guar

RAG lm_harness run Llama_2_7B:

In [ ]:
# 81.87 ->

# !lm_eval --model hf \
#     --model_args pretrained=$Llama_2_7B \
#     --include_path $INCLUDE_TASK_PATH \
#     --tasks $PARTIAL_TASK_PROMPT_ENG \
#     --retrieval_file "{RETRIEVED_DOCS}" \
#     --concat_k 1 \
#     --output "{llama_2_7b_RAG_results_partial_1st_path}" \
#     --log_samples

2024-11-15 14:18:07.959590: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 14:18:07.977258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 14:18:07.998397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 14:18:08.004816: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 14:18:08.020428: I tensorflow/core/platform/cpu_feature_guar

RAG lm_harness run Llama_3_3B:

In [ ]:
# !lm_eval --model hf \
#     --model_args pretrained=$Llama_3_3B \
#     --include_path $INCLUDE_TASK_PATH \
#     --tasks $PARTIAL_TASK_PROMPT_ENG \
#     --retrieval_file "{RETRIEVED_DOCS}" \
#     --concat_k 1 \
#     --output "{llama_3_3b_RAG_results_partial_1st_path}" \
#     --log_samples

2024-11-15 14:13:20.443965: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 14:13:20.461543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 14:13:20.483891: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 14:13:20.490720: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 14:13:20.506778: I tensorflow/core/platform/cpu_feature_guar

RAG lm_harness rus Ministral_3B:

In [ ]:
# !lm_eval --model hf \
#     --model_args pretrained=$Ministral_3B \
#     --include_path $INCLUDE_TASK_PATH \
#     --tasks $PARTIAL_TASK_PROMPT_ENG \
#     --retrieval_file "{RETRIEVED_DOCS}" \
#     --concat_k 3 \
#     --output "{ministral_3B_RAG_results_partial_1st_path}" \
#     --log_samples

2024-11-15 14:59:29.088905: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 14:59:29.107543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 14:59:29.129047: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 14:59:29.135594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 14:59:29.151308: I tensorflow/core/platform/cpu_feature_guar

# **Visualization:**

In [ ]:
import sys
module_path = '/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /harness_eval_large_scale'
if module_path not in sys.path:
    sys.path.append(module_path)
from lm_harness_visualization import LmHarnessVis
import numpy as np



comment** - in this line:
json_llama_2_RAG = llama_2_RAG_vis.read_res_json("********")
change the "*****" to the .jsno file name (same as without RAG)

Orca_2 visualization:

In [ ]:
# No RAG visualization Orca_2:
# orca_2_no_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Orca_2_7B/no_RAG_results/partial_1st_path/microsoft__Orca-2-7b")
# json_orca_2_no_RAG = orca_2_no_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T10-42-50.239223.json")

orca_2_no_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Orca_2_7B/no_RAG_results/partial_5th_path/microsoft__Orca-2-7b")
json_orca_2_no_RAG = orca_2_no_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T09-11-45.281239.json")



orca_2_no_RAG_df = orca_2_no_RAG_vis.create_res_df(json_orca_2_no_RAG)
orca_2_no_RAG_sub_acc_sub_df = orca_2_no_RAG_vis.calculate_accuracy(orca_2_no_RAG_df)
total_acc_orca_2_no_RAG = np.round(orca_2_no_RAG_vis.total_accuracy(orca_2_no_RAG_df),2)
orca_2_no_RAG_vis.create_bar_chart(orca_2_no_RAG_df,f'Orca_2_no_rag total acc: {total_acc_orca_2_no_RAG}',orca_2_no_RAG_sub_acc_sub_df)
# orca_2_no_RAG_vis.plot_average_likelihood(orca_2_no_RAG_df,"Orca_2_no_rag")
# orca_2_no_RAG_vis.plot_confidence_accuracy_correlation(orca_2_no_RAG_df,"Confident to Accuracy correlation - Orca_2 no RAG")

# RAG visualization Orca_2:
# orca_2_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Orca_2_7B/RAG_results/partial_1st_path/microsoft__Orca-2-7b")
# json_orca_2_RAG = orca_2_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-14T16-42-02.056195.json")

orca_2_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Orca_2_7B/RAG_results/partial_5th_path/microsoft__Orca-2-7b")
json_orca_2_RAG = orca_2_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-14T16-49-23.878589.json")

# orca_2_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Orca_2_7B/RAG_results/custom_corpus")
# json_orca_2_RAG = orca_2_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T11-04-38.044703.json")

orca_2_df_RAG = orca_2_RAG_vis.create_res_df(json_orca_2_RAG)
orca_2_RAG_sub_acc_sub_df = orca_2_RAG_vis.calculate_accuracy(orca_2_df_RAG)
total_acc_orca_2_RAG = np.round(orca_2_RAG_vis.total_accuracy(orca_2_df_RAG),2)
orca_2_RAG_vis.create_bar_chart(orca_2_df_RAG,f'Orca_2_RAG total acc: {total_acc_orca_2_RAG}',orca_2_RAG_sub_acc_sub_df)
orca_2_RAG_vis.plot_average_likelihood(orca_2_df_RAG,"Orca_2_RAG")
orca_2_RAG_vis.plot_confidence_accuracy_correlation(orca_2_df_RAG,"Confident to Accuracy correlation - Orca_2 RAG")

# RAG vs No RAG comparison:
orca_2_no_RAG_vis.create_comparison_bar_chart(orca_2_no_RAG_sub_acc_sub_df,orca_2_RAG_sub_acc_sub_df,"Orca_2_no_RAG","Orca_2_RAG")
orca_2_no_RAG_vis.plot_compare_likelihood(orca_2_no_RAG_df,orca_2_df_RAG,"Orca_2_no_RAG","Orca_2_RAG")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mistral_7B visualization:

In [ ]:
# No RAG visualization Mistral_7B:
# mistral_7b_no_RAG_vis = LmHarnessVis(mistral_7b_no_RAG_results_path+"/mistralai__Mistral-7B-v0.1")
# mistral_7b_no_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Mistral_7B/no_RAG_results/partial_1st_path/mistralai__Mistral-7B-v0.1")
# json_mistral_7b_no_RAG = mistral_7b_no_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T10-22-02.958834.json")

mistral_7b_no_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Mistral_7B/no_RAG_results/partial_5th_path/mistralai__Mistral-7B-v0.1")
json_mistral_7b_no_RAG = mistral_7b_no_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T15-21-35.447939.json")


mistral_7b_no_RAG_df = mistral_7b_no_RAG_vis.create_res_df(json_mistral_7b_no_RAG)
mistral_7b_no_RAG_sub_acc_sub_df = mistral_7b_no_RAG_vis.calculate_accuracy(mistral_7b_no_RAG_df)
total_acc_mistral_7b_no_RAG = np.round(mistral_7b_no_RAG_vis.total_accuracy(mistral_7b_no_RAG_df),2)
mistral_7b_no_RAG_vis.create_bar_chart(mistral_7b_no_RAG_df,f'Mistral_7B_no_rag total acc: {total_acc_mistral_7b_no_RAG}',mistral_7b_no_RAG_sub_acc_sub_df)
# mistral_7b_no_RAG_vis.plot_average_likelihood(mistral_7b_no_RAG_df,"Mistral_7B_no_rag")
# mistral_7b_no_RAG_vis.plot_confidence_accuracy_correlation(mistral_7b_no_RAG_df,"Confident to Accuracy correlation - Mistral_7B no RAG")

# RAG visualization Mistral_7B:
# mistral_7b_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Mistral_7B/RAG_results/partial_1st_path/mistralai__Mistral-7B-v0.1")
# json_mistral_7b_RAG = mistral_7b_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T14-35-20.834771.json")

mistral_7b_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Mistral_7B/RAG_results/partial_5th_path/mistralai__Mistral-7B-v0.1")
json_mistral_7b_RAG = mistral_7b_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T15-41-08.482236.json")

# mistral_7b_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Mistral_7B/RAG_results/custom_corpus")
# json_mistral_7b_RAG = mistral_7b_RAG_vis.read_res_json("samples_partial_sysengbench_2024-11-14T10-31-14.641784.json")

mistral_7b_df_RAG = mistral_7b_RAG_vis.create_res_df(json_mistral_7b_RAG)
mistral_7b_RAG_sub_acc_sub_df = mistral_7b_RAG_vis.calculate_accuracy(mistral_7b_df_RAG)
total_acc_mistral_7b_RAG = np.round(mistral_7b_RAG_vis.total_accuracy(mistral_7b_df_RAG),2)
mistral_7b_RAG_vis.create_bar_chart(mistral_7b_df_RAG,f'Mistral_7B_RAG total acc: {total_acc_mistral_7b_RAG}',mistral_7b_RAG_sub_acc_sub_df)
mistral_7b_RAG_vis.plot_average_likelihood(mistral_7b_df_RAG,"Mistral_7B_RAG")
mistral_7b_RAG_vis.plot_confidence_accuracy_correlation(mistral_7b_df_RAG,"Confident to Accuracy correlation - Mistral_7B RAG")

# RAG vs No RAG comparison:
mistral_7b_no_RAG_vis.create_comparison_bar_chart(mistral_7b_no_RAG_sub_acc_sub_df,mistral_7b_RAG_sub_acc_sub_df,"Mistral_7B_no_rag","Mistral_7B_rag")
mistral_7b_no_RAG_vis.plot_compare_likelihood(mistral_7b_no_RAG_df,mistral_7b_df_RAG,"Mistral_7B_no_rag","Mistral_7B_rag")







Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Llama_2_13B Visualization:

In [ ]:
# No RAG visualization Llama_2_13B:
# llama_2_13b_no_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_2_13B/no_RAG_results/partial_1st_path/meta-llama__Llama-2-13b-chat-hf")
# json_llama_2_13_no_RAG = llama_2_13b_no_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-14T16-06-25.251975.json")

llama_2_13b_no_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_2_13B/no_RAG_results/partial_5th_path/meta-llama__Llama-2-13b-chat-hf")
json_llama_2_13_no_RAG = llama_2_13b_no_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T15-26-46.107117.json")


llama_2_13_no_RAG_df = llama_2_13b_no_RAG_vis.create_res_df(json_llama_2_13_no_RAG)
llama_2_13_no_RAG_sub_acc_sub_df = llama_2_13b_no_RAG_vis.calculate_accuracy(llama_2_13_no_RAG_df)
total_acc_llama_2_13_no_RAG = np.round(llama_2_13b_no_RAG_vis.total_accuracy(llama_2_13_no_RAG_df),2)
llama_2_13b_no_RAG_vis.create_bar_chart(llama_2_13_no_RAG_df,f'Llama_2_13B_no_rag total acc: {total_acc_llama_2_13_no_RAG}',llama_2_13_no_RAG_sub_acc_sub_df)
# llama_2_13b_no_RAG_vis.plot_average_likelihood(llama_2_13_no_RAG_df,"Llama_2_13B_no_rag")
# llama_2_13b_no_RAG_vis.plot_confidence_accuracy_correlation(llama_2_13_no_RAG_df,"Confident to Accuracy correlation - Llama_2_13B no RAG")


# RAG visualization Llama_2_13B:
# llama_2_13b_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_2_13B/RAG_results/partial_1st_path/meta-llama__Llama-2-13b-chat-hf")
# json_llama_2_13_RAG = llama_2_13b_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T12-59-30.220476.json")

llama_2_13b_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_2_13B/RAG_results/partial_5th_path/meta-llama__Llama-2-13b-chat-hf")
json_llama_2_13_RAG = llama_2_13b_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T15-38-45.114743.json")

# llama_2_13b_RAG_vis = LmHarnessVis("/content/drive/MyDrive/Technion - LLM Research 2024/Coding Environment /Resources/RAG resouces/lm_harness_results/Llama_2_13B/RAG_results/custom_corpus_partial")
# json_llama_2_13_RAG = llama_2_13b_RAG_vis.read_res_json("samples_partial_sysengbench_prompt_eng_2024-11-15T11-19-57.314650.json")

llama_2_13_df_RAG = llama_2_13b_RAG_vis.create_res_df(json_llama_2_13_RAG)
llama_2_13_RAG_sub_acc_sub_df = llama_2_13b_RAG_vis.calculate_accuracy(llama_2_13_df_RAG)
total_acc_llama_2_13_RAG = np.round(llama_2_13b_RAG_vis.total_accuracy(llama_2_13_df_RAG),2)
llama_2_13b_RAG_vis.create_bar_chart(llama_2_13_df_RAG,f'Llama_2_13B_RAG total acc: {total_acc_llama_2_13_RAG}',llama_2_13_RAG_sub_acc_sub_df)
llama_2_13b_RAG_vis.plot_average_likelihood(llama_2_13_df_RAG,"Llama_2_13B_RAG")
llama_2_13b_RAG_vis.plot_confidence_accuracy_correlation(llama_2_13_df_RAG,"Confident to Accuracy correlation - Llama_2_13B RAG")

# RAG vs No RAG comparison:
llama_2_13b_RAG_vis.create_comparison_bar_chart(llama_2_13_no_RAG_sub_acc_sub_df,llama_2_13_RAG_sub_acc_sub_df,"Llama_2_13B_no_rag","Llama_2_13B_rag")
llama_2_13b_RAG_vis.plot_compare_likelihood(llama_2_13_no_RAG_df,llama_2_13_df_RAG,"Llama_2_13B_no_rag","Llama_2_13B_rag")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Models Comparison:

In [ ]:

vis = LmHarnessVis("")
fig1 = vis.create_comparison_table(llama_2_13_no_RAG_df, orca_2_no_RAG_df, mistral_7b_no_RAG_df, "llama_2_13b", "Orca_2", "Mistral_7B", "Model Comparison Table NO RAG (start at 5th)")
fig2 = vis.create_comparison_table(llama_2_13_df_RAG, orca_2_df_RAG, mistral_7b_df_RAG, "llama_2_13b", "Orca_2", "Mistral_7B", "Model Comparison Table RAG (start at 5th)")

fig1.write_html("/content/comparison_table_no_RAG_5th.html")
fig2.write_html("/content/comparison_table_RAG_5th.html")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ministral_8B Visualization:

In [ ]:
# #No RAG visualization:
# ministral_8_no_RAG_vis = LmHarnessVis()
# json_ministral_8_no_RAG = mistral_8_no_RAG_vis.read_res_json()
# ministral_8_df_no_RAG = mistral_8_no_RAG_vis.create_res_df(json_ministral_8_no_RAG)
# ministral_8_no_RAG_sub_acc_sub_df = mistral_8_no_RAG_vis.calculate_accuracy(ministral_8_df_no_RAG)
# total_acc_ministral_8_no_RAG = np.round(mistral_8_no_RAG_vis.total_accuracy(ministral_8_df_no_RAG),2)
# ministral_8_no_RAG_vis.create_bar_chart(ministral_8_df_no_RAG,f"Ministral_8B_no_RAG total acc: {total_acc_ministral_8_no_RAG}",ministral_8_no_RAG_sub_acc_sub_df)
# ministral_8_no_RAG_vis.plot_average_likelihood(ministral_8_df_no_RAG,"Ministral_8B_no_RAG")
# ministral_8_no_RAG_vis.plot_confidence_accuracy_correlation(ministral_8_df_no_RAG,"Confident to Accuracy correlation - Mistral_8B")

# #RAG visualization:
# ministral_8_RAG_vis = LmHarnessVis()
# json_ministral_8_RAG = mistral_8_RAG_vis.read_res_json()
# ministral_8_df_RAG = mistral_8_RAG_vis.create_res_df(json_ministral_8_RAG)
# ministral_8_RAG_sub_acc_sub_df = mistral_8_RAG_vis.calculate_accuracy(ministral_8_df_RAG)
# total_acc_ministral_8_RAG = np.round(mistral_8_RAG_vis.total_accuracy(ministral_8_df_RAG),2)
# ministral_8_RAG_vis.create_bar_chart(ministral_8_df_RAG,f"Ministral_8B_RAG total acc: {total_acc_ministral_8_RAG}",ministral_8_RAG_sub_acc_sub_df)
# ministral_8_RAG_vis.plot_average_likelihood(ministral_8_df_RAG,"Ministral_8B_RAG")

# # RAG vs No RAG comparison:
# ministral_8_no_RAG_vis.create_comparison_bar_chart(ministral_8_no_RAG_sub_acc_sub_df,ministral_8_RAG_sub_acc_sub_df,"Ministral_8B_no_RAG","MInistral_8b_RAG")
# ministral_8_no_RAG_vis.plot_compare_likelihood(ministral_8_df_no_RAG,ministral_8_df_RAG,"Ministral_8B_no_RAG","MInistral_8b_RAG")




Llama_2_7B visualization

In [ ]:
# # No RAG visualization Llama_2_7B:
# llama_2_no_RAG_vis = LmHarnessVis(llama_2_7b_no_RAG_results_path+"/meta-llama__Llama-2-7b-chat-hf")
# json_llama_2_no_RAG = llama_2_no_RAG_vis.read_res_json("samples_demo_sysengbench_2024-11-04T11-14-25.959726.json")
# llama_2_df_no_RAG = llama_2_no_RAG_vis.create_res_df(json_llama_2_no_RAG)
# llama_2_no_RAG_sub_acc_sub_df = llama_2_no_RAG_vis.calculate_accuracy(llama_2_df_no_RAG)
# total_acc_llama_2_no_RAG = np.round(llama_2_no_RAG_vis.total_accuracy(llama_2_df_no_RAG), 2)
# llama_2_no_RAG_vis.create_bar_chart(llama_2_df_no_RAG, f'Llama 2_no_rag total acc: {total_acc_llama_2_no_RAG}', llama_2_no_RAG_sub_acc_sub_df)
# llama_2_no_RAG_vis.plot_average_likelihood(llama_2_df_no_RAG, "Llama 2_no_rag")
# llama_2_no_RAG_vis.plot_confidence_accuracy_correlation(llama_2_df_no_RAG,"Confident to Accuracy correlation - Llama_2_7B")

# # RAG visualization Llama_2_7B:
# # llama_2_RAG_vis = LmHarnessVis(llama_2_7b_RAG_results_path+"/meta-llama__Llama-2-7b-chat-hf")
# # json_llama_2_RAG = llama_2_RAG_vis.read_res_json("samples_demo_sysengbench_2024-11-07T10-00-09.899169.json")
# llama_2_RAG_vis = LmHarnessVis("/content/meta-llama__Llama-2-7b-chat-hf")
# json_llama_2_RAG = llama_2_RAG_vis.read_res_json("samples_demo_sysengbench_2024-11-08T11-50-51.506410.json")
# llama_2_df_RAG = llama_2_RAG_vis.create_res_df(json_llama_2_RAG)
# llama_2_RAG_sub_acc_sub_df = llama_2_RAG_vis.calculate_accuracy(llama_2_df_RAG)
# total_acc_llama_2_RAG = np.round(llama_2_RAG_vis.total_accuracy(llama_2_df_RAG), 2)
# llama_2_RAG_vis.create_bar_chart(llama_2_df_RAG, f'Llama 2_RAG total acc: {total_acc_llama_2_RAG}', llama_2_RAG_sub_acc_sub_df)
# llama_2_RAG_vis.plot_average_likelihood(llama_2_df_RAG, "Llama 2_RAG")

# # RAG vs No RAG comparison:
# llama_2_no_RAG_vis.create_comparison_bar_chart(llama_2_no_RAG_sub_acc_sub_df,llama_2_RAG_sub_acc_sub_df,"Llama_2_7B_no_RAG","Llama_2_7B_RAG")
# llama_2_no_RAG_vis.plot_compare_likelihood(llama_2_df_no_RAG,llama_2_df_RAG,"Llama_2_7B_no_RAG","Llama_2_7B_RAG")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Llama_3_3B Visualization:

In [ ]:
# # No RAG visualization Llama_3_3B:
# llama_3_3b_no_RAG_vis = LmHarnessVis(llama_3_3b_no_RAG_results_path+"/meta-llama__Llama-3.2-3B")
# json_llama_3_no_RAG = llama_3_3b_no_RAG_vis.read_res_json("samples_demo_sysengbench_2024-11-02T09-54-16.144188.json")
# llama_3_no_RAG_df = llama_3_3b_no_RAG_vis.create_res_df(json_llama_3_no_RAG)
# llama_3_no_RAG_sub_acc_sub_df = llama_3_3b_no_RAG_vis.calculate_accuracy(llama_3_no_RAG_df)
# total_acc_llama_3_no_RAG = np.round(llama_3_3b_no_RAG_vis.total_accuracy(llama_3_no_RAG_df),2)
# llama_3_3b_no_RAG_vis.create_bar_chart(llama_3_no_RAG_df,f'Llama_3_3B_no_rag total acc: {total_acc_llama_3_no_RAG}',llama_3_no_RAG_sub_acc_sub_df)
# llama_3_3b_no_RAG_vis.plot_average_likelihood(llama_3_no_RAG_df,"Llama_3_3B_no_rag")
# llama_3_3b_no_RAG_vis.plot_confidence_accuracy_correlation(llama_3_no_RAG_df,"Confident to Accuracy correlation - Llama_3_3B")


# # RAG visualization Llama_3_3B:
# llama_3_3b_RAG_vis = LmHarnessVis(llama_3_3b_RAG_results_path+"/meta-llama__Llama-3.2-3B")
# json_llama_3_RAG = llama_3_3b_RAG_vis.read_res_json("samples_demo_sysengbench_2024-11-07T07-27-29.359722.json")
# llama_3_df_RAG = llama_3_3b_RAG_vis.create_res_df(json_llama_3_RAG)
# llama_3_RAG_sub_acc_sub_df = llama_3_3b_RAG_vis.calculate_accuracy(llama_3_df_RAG)
# total_acc_llama_3_RAG = np.round(llama_3_3b_RAG_vis.total_accuracy(llama_3_df_RAG),2)
# llama_3_3b_RAG_vis.create_bar_chart(llama_3_df_RAG,f'Llama_3_3B_RAG total acc: {total_acc_llama_3_RAG}',llama_3_RAG_sub_acc_sub_df)
# llama_3_3b_RAG_vis.plot_average_likelihood(llama_3_df_RAG,"Llama_3_3B_RAG")

# # RAG vs No RAG comparison:
# llama_3_3b_no_RAG_vis.create_comparison_bar_chart(llama_3_no_RAG_sub_acc_sub_df,llama_3_RAG_sub_acc_sub_df,"Llama_3_3B_no_rag","Llama_3_3B_rag")
# llama_3_3b_no_RAG_vis.plot_compare_likelihood(llama_3_no_RAG_df,llama_3_df_RAG,"Llama_3_3B_no_rag","Llama_3_3B_rag")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
